In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

In [3]:
os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY

In [4]:
from langchain.document_loaders import PyPDFLoader

In [5]:
%pwd

'g:\\[AI Agent Development]\\Question-and-Answer-Generator-App\\research'

In [6]:
%cd ..

g:\[AI Agent Development]\Question-and-Answer-Generator-App


In [7]:
%pwd

'g:\\[AI Agent Development]\\Question-and-Answer-Generator-App'

In [8]:
file_path = "data/SDG.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [9]:
data

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Macintosh)', 'creationdate': '2017-12-26T16:03:25-05:00', 'moddate': '2017-12-27T15:29:10-05:00', 'trapped': '/False', 'source': 'data/SDG.pdf', 'total_pages': 24, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Macintosh)', 'creationdate': '2017-12-26T16:03:25-05:00', 'moddate': '2017-12-27T15:29:10-05:00', 'trapped': '/False', 'source': 'data/SDG.pdf', 'total_pages': 24, 'page': 1, 'page_label': '2'}, page_content=''),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Macintosh)', 'creationdate': '2017-12-26T16:03:25-05:00', 'moddate': '2017-12-27T15:29:10-05:00', 'trapped': '/False', 'source': 'data/SDG.pdf', 'total_pages': 24, 'page': 2, 'page_label': '3'}, page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE T

In [11]:
len(data)

24

*Used for loop to get the page content only*

In [12]:
question_gen = ""
for page in data:
    question_gen += page.page_content

In [13]:
question_gen

'IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence that we can succeed. In the past 15 yea

*Import required Library* <br>
- AutoTokenizer (from Hugging Face) automatically loads the correct tokenizer based on a model name.
- TokenTextSplitter (from LangChain) splits long texts into manageable chunks based on tokens, which is important for LLMs with context/token limits.


In [14]:
from transformers import AutoTokenizer
from langchain.text_splitter import TokenTextSplitter

c:\Users\HP\anaconda3\envs\QAgenerator\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- Loads the tokenizer for the Mistral 7B Instruct model from Hugging Face.
- This tokenizer understands how Mistral breaks down input text into tokens.
- It requires internet access the first time and downloads the tokenizer files to your local machine

In [15]:
# Load Mistral tokenizer from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    use_fast=False
    )

-  tokenizer=tokenizer, means you're passing a variable named tokenizer as a parameter named tokenizer.
- chunk_size=10000: Each chunk will have up to 10,000 tokens.
- chunk_overlap=200: 200 tokens overlap between chunks to preserve context.

In [16]:
splitter_ques_gen = TokenTextSplitter.from_huggingface_tokenizer(
    tokenizer = tokenizer,
    chunk_size = 10000,
    chunk_overlap = 200
)

In [17]:
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)

In [18]:
chunk_ques_gen

['IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence that we can succeed. In the past 15 ye

In [19]:
len(chunk_ques_gen)

1

In [20]:
type(chunk_ques_gen[0])

str

*Now we will rechunk* <br>
Our document may large, hence we will rechunk the chunked document.
Since chunk contain string, so we have to convert it into document

In [21]:
from langchain.docstore.document import Document

In [22]:
document_ques_gen = [Document(page_content=t) for t in chunk_ques_gen]

In [23]:
document_ques_gen

[Document(metadata={}, page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence tha

In [24]:
splitter_ans_gen = TokenTextSplitter.from_huggingface_tokenizer(
    tokenizer = tokenizer,
    chunk_size = 1000,
    chunk_overlap = 100
)

In [25]:
document_answer_gen = splitter_ans_gen.split_documents(document_ques_gen)

In [26]:
document_answer_gen

[Document(metadata={}, page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence tha

In [27]:
len(document_answer_gen)

5

*Now we will generate our questions*

In [44]:
from langchain.chat_models import ChatOpenAI # If you use open AI API
#from langchain_openrouter import ChatOpenRouter

In [45]:
llm_ques_gen_pipeline = ChatOpenAI(
    model_name = 'mistralai/Mistral-7B-Instruct',
    temperature = 0.3,
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1"
)

In [46]:
prompt_template = """ 
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

----------------
{text}
----------------

Create questions that will prepare the coders or programers for their tests.
Make sure not to lose any important information.

QUESTION:
"""

In [47]:
from langchain.prompts import PromptTemplate

In [48]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [49]:
refined_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.

--------------
{text}
--------------

Given the new context, refining the original questions in English.
If the context is not helpful, please provide the original questions.

QUESTIONS:
""")

In [50]:
REFINED_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refined_template,
)

In [74]:
from langchain.chains.summarize import load_summarize_chain

In [ ]:
ques_gen_chain = load_summarize_chain(llm = llm_ques_gen_pipeline,
                                      chain_type="refine",
                                      verbose=True,
                                      question_prompt = PROMPT_QUESTIONS,
                                      refine_prompt=REFINED_PROMPT_QUESTIONS)

In [52]:
document_ques_gen

[Document(metadata={}, page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence tha

In [53]:
ques = ques_gen_chain.run(document_ques_gen)

print(ques)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

----------------
IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD 
CAME TOGETHER TO FACE THE FUTURE.
And what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. 
Not just in some faraway place, but in their own cities and towns and villages.
They knew things didn’t have to be this way. They knew we had enough 
food to feed the world, but that it wasn’t getting shared. They knew there 
were medicines for HIV and other diseases, but they cost a lot. They knew 
that earthquakes and floods were inevitable, but that the high death 
tolls were not. 
They also knew that billions of people worldwide shared their hope for a 
better future.
So leaders

In [54]:
from langchain_huggingface import HuggingFaceEmbeddings


In [55]:
#embeddings = OpenAIEmbeddings() #This is for open AI API

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [56]:
from langchain.vectorstores import FAISS

In [57]:
vector_store = FAISS.from_documents(document_answer_gen, embeddings)

In [59]:
llm_answer_gen = ChatOpenAI(
    model_name = 'mistralai/Mistral-7B-Instruct',
    temperature = 0.1,
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1"
)

In [60]:
ques

' 1. What is the name of the plan created by leaders from 193 countries in 2015 to address global issues such as poverty, hunger, and climate change?\n   Answer: Sustainable Development Goals (SDGs)\n\n2. What is the goal of the SDGs in relation to extreme poverty?\n   Answer: To end extreme poverty in all forms by 2030.\n\n3. How many people around the world still live on less than $1.25 a day?\n   Answer: More than 800 million people\n\n4. What is the goal of the SDG related to hunger and nutrition?\n   Answer: To end hunger and malnutrition once and for all.\n\n5. What is the goal of the SDG related to health and well-being?\n   Answer: To ensure healthy lives and promote well-being for all at all ages.\n\n6. What is the goal of the SDG related to education?\n   Answer: To ensure inclusive and equitable quality education and promote lifelong learning opportunities for all.\n\n7. What is the goal of the SDG related to gender equality and empowerment of women and girls?\n   Answer: To

In [61]:
ques_list = ques.split("\n")

In [62]:
ques_list

[' 1. What is the name of the plan created by leaders from 193 countries in 2015 to address global issues such as poverty, hunger, and climate change?',
 '   Answer: Sustainable Development Goals (SDGs)',
 '',
 '2. What is the goal of the SDGs in relation to extreme poverty?',
 '   Answer: To end extreme poverty in all forms by 2030.',
 '',
 '3. How many people around the world still live on less than $1.25 a day?',
 '   Answer: More than 800 million people',
 '',
 '4. What is the goal of the SDG related to hunger and nutrition?',
 '   Answer: To end hunger and malnutrition once and for all.',
 '',
 '5. What is the goal of the SDG related to health and well-being?',
 '   Answer: To ensure healthy lives and promote well-being for all at all ages.',
 '',
 '6. What is the goal of the SDG related to education?',
 '   Answer: To ensure inclusive and equitable quality education and promote lifelong learning opportunities for all.',
 '',
 '7. What is the goal of the SDG related to gender equa

In [ ]:
from langchain.chains import RetrievalQA

In [68]:
answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen,
                                                      chain_type="stuff",
                                                      retriever=vector_store.as_retriever())

In [73]:
# Answer each question and save to a file
for question in ques_list:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("-------------------------------------\\n\\n")
    #Save answer to a file
    with open("answer.txt", "a") as f:
        f.write("\nQuestion: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("-------------------------------------\\n\\n")

Question:   1. What is the name of the plan created by leaders from 193 countries in 2015 to address global issues such as poverty, hunger, and climate change?
Answer:   The plan created by leaders from 193 countries in 2015 to address global issues such as poverty, hunger, and climate change is called the Sustainable Development Goals (SDGs).
-------------------------------------\n\n
Question:     Answer: Sustainable Development Goals (SDGs)
Answer:   The Sustainable Development Goals (SDGs) are a set of 17 goals created by leaders from 193 countries in 2015. These goals aim to end poverty, hunger, and promote sustainable development by 2030. The SDGs include goals such as ending extreme poverty, ensuring healthy lives and promoting well-being for all, ensuring inclusive and equitable quality education, and taking urgent action to combat climate change. The United Nations Development Programme (UNDP) is one of the leading organizations working to fulfill the SDGs.
--------------------